In [1]:
import numpy as np

In [2]:
# Had a little help as always
# https://www.geeksforgeeks.org/dice-throw-problem/

# Here is the table of correct answers for 6 sided dice, to check accuracy
# https://wizardofodds.com/gambling/dice/2/
def find_ways(k,n,x):
    if x<1: return 0
    if n==1: return int(x<=k)#1 if true else 0
    num_ways = 0
    for i in range(1,k+1):
        num_ways += find_ways(k,n-1,x-i)
    return num_ways

In [3]:
find_ways(6,5,14)

540

In [4]:
'''
Dynamic Solution -- translated to numpy
'''
def findWays(k,n,x):
    # The extra row/col make indexing easier, but could be removed (it's not really too bad tho)
    table = np.zeros((n+1,x+1), dtype=np.uint64)
    
    if k < x:
        table[1][1:k+1] = 1
    else:
        table[1][1:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1

                
    print(table)
    return table[n][x]

    

In [5]:
# findWays(6,6,6)


In [6]:
def get_freq(k,n):
    x = n*k
    table = np.zeros((n+1,x+1), dtype = np.float64)
    
    if k < x:
        table[1][:k+1] = 1
    else:
        table[1][:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            #table[i][j] = table[i][j-1] + table[i-1][j-1]
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1
#    print(table)
    
    return table[n]

In [7]:
get_freq(6,3)

array([  0.,   0.,   0.,   1.,   3.,   6.,  10.,  15.,  21.,  25.,  27.,
        27.,  25.,  21.,  15.,  10.,   6.,   3.,   1.])

In [8]:
def get_list_freq(dice):
    #setup
    #there are n dice, like last time
    n = dice.size
    # x needs to be the upper bound, which would be rolling k on all dice
    x = np.sum(dice)
    # init the table
    table = np.zeros((n+1,x+1), dtype=np.uint64)
    
    dice = sorted(dice)[::-1]
    
    if np.max(dice) < x:
        table[1][:np.max(dice)+1] = 1
    else:
        table[1][:x+1] = 1
    
    i = 1
    for k in dice:
        # we need to handle each dice face independently
        #might need temporary bound for 2nd loop
        for j in range(2,x+1):
            l = 1
            while l<j and l <=k:
                table[i][j] += table[i-1][j-l]
                l+=1   
        i += 1
#    print(table)
    return table[n]

In [22]:
print(get_list_freq(np.array([6,6,3])))
print(get_freq(6,3))

[ 0  0  0  1  3  6  9 12 15 16 15 12  9  6  3  1]
[  0.   0.   0.   1.   3.   6.  10.  15.  21.  25.  27.  27.  25.  21.  15.
  10.   6.   3.   1.]


In [10]:
def get_dict_freq(some_dict):
    lis = []
    for key in some_dict:
        lis += [key]*some_dict[key]
    to_call = np.asarray(lis)
    # print(to_call)
    return get_list_freq(to_call)

In [11]:
test = {6:2, 4:1}
print(get_dict_freq(test))
print(get_list_freq(np.array([6,6,4])))

[ 0  0  0  1  3  6 10 14 18 20 20 18 14 10  6  3  1]
[ 0  0  0  1  3  6 10 14 18 20 20 18 14 10  6  3  1]


In [12]:
# np.random.choice(5000,200,replace=False)

In [13]:
'''
Attempting to use linear algebra to solve get_freq(6,3), then will build from there

size = nk-n+1
'''
size = 3*6-3+1


Y = np.ones((size,size))
Y = np.triu(Y)

Y = np.triu(np.flip(Y,1))

'''
I also need to initialize the starting array, a 1xSize arry with the initial frrequencies
'''

a = np.zeros(size)
a[0:6] = 1
#print(a)
print('The transformation matrix A:')
# print(Y)

for i in range(size-2*6):
    Y[i][(6+i):size-(6+i)] = 0
    

print(Y)

print(a)

print(np.linalg.matrix_power(Y.T,2).dot(a))


The transformation matrix A:
[[ 1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  1.  1.  0.]
 [ 0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.

In [14]:
def freq(k,n):
    size = k*n-n+1
    # print(size)
    X = np.ones((size,size))
    X = np.triu(X)
    X = np.triu(np.flip(X,1))
    for i in range(size-2*k):
        X[i][k+i:size-(k+i)] = 0
    print(X.T)
    a = np.zeros(size)
    a[0:k] = 1
    print(a)
    return np.linalg.matrix_power(X.T,n-1).dot(a)
    #return X

In [15]:
result = get_freq(6, 300)
# %timeit get_freq(6, 300)
result[(300*5)//2+300]

3.7588197132593194e+231

In [16]:
res = freq(6, 3)
# %timeit freq(6, 300)
#res[(300*5)//2]
print(res)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [49]:
#moving on to the tougher version
def freq_dict(some_dict):
    # Faces is just the set of different faces stored in a list
    faces = sorted(some_dict.keys())[::-1]
    size = sum(key*some_dict[key] for key in some_dict) - sum(some_dict[key] for key in some_dict) + 1
    # freq_matrix will store the results, I will keep it stored in the same place 
    freq_matrix = np.zeros(size)
    freq_matrix[:faces[0]] = 1 
    # The transformation matrix will change slightly for each face 
    # To start, initialize to the largest die number and we will adjust as we go
    A = np.ones((size,size))
    A = np.triu(A)
    A = np.triu(np.flip(A,1))
    for i in range(size-2*faces[0]):
        A[i][faces[0]+i:size-(faces[0]+i)] = 0
    
    # Handle the first size dice to get the ball rollin
    if some_dict[faces[0]] > 1:
        freq_matrix = np.linalg.matrix_power(A.T,some_dict[faces[0]]-1).dot(freq_matrix)
    
    # Now handle the rest of the dice
    for face in faces[1:]:
        for i in range(size-2*face):
            A[i][face+i:size-(face+i)] = 0
        freq_matrix = np.linalg.matrix_power(A.T,some_dict[face]).dot(freq_matrix)
    
    return freq_matrix
    

In [56]:
test = {6:2,4:2,3:3}
print(freq_dict(test))

[  1.00000000e+00   7.00000000e+00   2.80000000e+01   8.10000000e+01
   1.87000000e+02   3.64000000e+02   6.17000000e+02   9.31000000e+02
   1.26800000e+03   1.57300000e+03   1.78700000e+03   1.86400000e+03
   1.78700000e+03   1.57300000e+03   1.26800000e+03   9.31000000e+02
   6.17000000e+02   3.64000000e+02   1.87000000e+02   8.10000000e+01
   2.80000000e+01   7.00000000e+00   1.00000000e+00]


In [54]:
get_dict_freq({6:2,4:2, 3:3})

array([   0,    0,    0,    0,    0,    0,    0,    1,    7,   28,   81,
        187,  364,  617,  931, 1268, 1573, 1787, 1864, 1787, 1573, 1268,
        931,  617,  364,  187,   81,   28,    7,    1], dtype=uint64)

In [295]:
size_x = 3+6+6-3+1
T = np.zeros((size_x,size_x))
T[0][:6] = 1
T[0][size_x-6:] = 1
T[1][1:size_x-1] = 1
T[2][3:10] = 1
T[3][4:9] = 1
T[4][5:8] = 1
T[5][6:7] = 1
#T[6][6:7] = 1
print(T)
A = np.zeros(size_x)
A[:6]=1
print(np.linalg.matrix_power(T.T,2).dot(A))

[[ 1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[  1.   3.   3.   5.   8.  12.  16.  12.   8.   5.   3.   3.   1.]
